#### 제주도 핫플레이스 인스타그램에서 크롤링하기

##### matplotlib 한글설정

##### 셀레니움 설정

In [9]:
import matplotlib
from matplotlib import pyplot as plt
# import matplotlib.pyplot as plt
from matplotlib import font_manager, rc, rcParams
import seaborn as sns

font_path = 'C:/Windows/Fonts/malgun.ttf'
font = font_manager.FontProperties(fname=font_path).get_name() # malgun
rc('font', family=font)
rcParams['axes.unicode_minus'] = False

In [25]:
# 셀레니움
# 모듈 임포트
from selenium import webdriver

driver = webdriver.Chrome()
url = 'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91/'
driver.get(url)

##### 첫번째 게시글 찾아서 클릭하기

In [39]:
from selenium.webdriver.common.by import By
import time

def select_first_post(driver):
    first = driver.find_elements(By.CLASS_NAME, '_aagw')[0]
    first.click()
    time.sleep(3)

select_first_post(driver)

##### 게시글 정보 가져오기

In [40]:
import re # RegularExpression 정규식 처리 모듈
from bs4 import BeautifulSoup
import unicodedata # 유니코드 데이터 처리 모듈

def get_content(driver):
    # 1. 데이터 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 2. 본문내용 찾아오기
    try:
        # content = soup.select('div._a9zr')[0].text
        content = soup.select('div._a9zr')[0].get_text(separator='|')  # _a9zr은 추후 변경됨(인스타그램 정책)
    except:
        content = ''

    # 3. 본문내용에서 해시태그 가져오기(정규식)
    tags = re.findall(r'#[^\s#,\\]+', content)

    # 4. 작성일자 정보
    data = soup.select('time._aaqe')[0]['datetime'][:10]

    # 5. 좋아요수 가져오기
    try:
        like = int(soup.select('span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp > span')[0].text)
    except:
        like = 0

    # 6. 위치정보
    # 정규식 학습필수
    try:
        regex = r'(\w+[시,읍]\s*)'\
                r'?(\w+\d*\w*[로,리,길]\s*)'\
                r'?(\w*\d+-?\d*)?'\

        place = re.findall(regex, content)
        for pl in place:
            if pl[0] != '' and pl[1] != '' and pl[2] != '':
                address = f'{pl[0]} {pl[1]} {pl[2]}'

    except:
        address = ''

    # 7. 정보 저장
    data = [content, data, like, address, tags]
    return data

get_content(driver)

['neriestadogs(협찬) @cafe_the_light_jeju💗제주도에서 가장 핫한 제주카페💗 🐋오션뷰 카페 ✔카페더라이트❝ 성산일출봉과 바다를 한 눈에 볼 수 있는 뷰맛집 ❞………………《 🍮 제주 카페더라이트 》…………………📍 제주 서귀포시 성산읍 한도로 269 카페더라이트 📍⏰ 매일 9:00 ~ 19:00📞 064-784-0080……………………………………………………………………………카페 전면 오션뷰 통창으로바다를 느낄 수 있는 "오션뷰카페"우도와 성산일출봉이 보이는 "루프탑 뷰맛집"인생샷을 남길 수 있는 포토존까지😍휴양지 느낌 가득 , 제주 느낌 가득한"야외좌석"도 있어요❣1층 2층 루프탑까지 넓은 카페만큼넓은 주차장까지👍✔️ 대표 메뉴1 "시그니처 라이트라떼(땅콩라떼)"✔️ 대표 메뉴2 "시그니처 제주아이스티"✔️ 대표 메뉴3 "한라봉 에이드"✔️디저트 ,베이커리 엄마손맛 수제!!웰빙 프리미엄 재료로 제주 특산품 한라봉을 이용한 디저트 !! “한라봉 몽블랑 ” 당근케익 베이커리 등 🎂 커플,친구,가족들과 함께 아름다운 제주를 느끼며소중한 시간을 보낼 수 있는 곳 💕……………………………………………………………………………#제주여행 #제주카페 #성산일출봉카페 #제주카페추천 #제주루프탑카페 #제주가볼만한곳 #제주여행코스 #제주도날씨 #제주서귀포카페추천 #제주가볼만한곳 #제주데이트 #제주도카페 #제주도맛집 #제주도 #카페22분',
 '2023-06-30',
 1063,
 '성산읍  한도로  269',
 ['#제주여행',
  '#제주카페',
  '#성산일출봉카페',
  '#제주카페추천',
  '#제주루프탑카페',
  '#제주가볼만한곳',
  '#제주여행코스',
  '#제주도날씨',
  '#제주서귀포카페추천',
  '#제주가볼만한곳',
  '#제주데이트',
  '#제주도카페',
  '#제주도맛집',
  '#제주도',
  '#카페22분']]

##### 다음 게시글 열기

In [41]:
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, ('._aaqg._aaqh'))
    right.find_element(By.XPATH, '//button')
    right.click()
    time.sleep(3)

move_next(driver)

#### 자동 크롤링 구현

In [52]:
# 모듈 임포트 생략

driver = webdriver.Chrome()
url = 'https://www.instagram.com'
driver.get(url)
time.sleep(5)
# select_first_post(driver)
# get_content(driver)
# move_next(driver)

In [53]:
def insta_login():
    id = '01092038196'
    pw = '비밀번호'
    input_id = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[0]
    input_id.clear()
    input_id.send_keys(id)
    input_pw = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[1]
    input_pw.clear()
    input_pw.send_keys(pw)
    input_pw.submit()
    time.sleep(2.0)
    
insta_login()

In [35]:
# 알림설정 버튼클릭
def btn_click():
    alarm_btn = driver.find_element(By.CSS_SELECTOR, '._a9--._a9_0')
    alarm_btn.click()
btn_click()

In [38]:

# 제주도 맛집 찾기 함수
def insta_searching(word):
    url = f'https://www.instagram.com/explore/tags/{word}'
    return url

driver.get(insta_searching('제주도맛집'))

In [48]:
# 크롤링 시작하기
driver = webdriver.Chrome()
url = 'https://www.instagram.com'
driver.get(url)
time.sleep(5)

insta_login()
time.sleep(1)
btn_click()

word = '제주도맛집'

url = insta_searching(word)
driver.get(url)
time.sleep(10.0)

select_first_post(driver)

result = []
count = 0

target = 5
for i in range(target):
    count += 1
    try:
        data = get_content(driver)
        result.append(data)
        move_next(driver)
    except:
        time.sleep(2.0)
        move_next(driver)
    print(count)

print(len(result))

1
2
3
4
5
1


In [49]:
import pandas as pd

results_df = pd.DataFrame(result)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('./1_crawling_jejudoHotplaces.xlsx', index=False)